# 2.4: Evaluating Hyperparameters
## Part 2: Deep Learning

### 1. Importing Libraries and Data

In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

In [47]:
# Create path variable for re-use when importing data
path = r'C:\Users\sorca\Documents\3ML-2_4\Task'

In [48]:
# Load climate dataframe
df_climate = pd.read_csv(os.path.join(path,'Climate_cleaned.csv'), index_col = False)

In [49]:
df_climate.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [50]:
df_climate.shape

(22950, 135)

In [51]:
# Load "pleasant" dataframe
df_pleasant = pd.read_csv(os.path.join(path,'Pleasant_cleaned.csv'), index_col = False)

In [52]:
df_pleasant.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
df_pleasant.shape

(22950, 15)

### 2. Reshaping Data for Modeling

In [55]:
# Turn X and answers from a df to arrays

X = np.array(df_climate)
y = np.array(df_pleasant)

In [56]:
X = X.reshape(-1,15,9)

In [57]:
X.shape

(22950, 15, 9)

In [58]:
# Use argmax to transform y

y =  np.argmax(y, axis = 1)

In [59]:
y.shape

(22950,)

In [60]:
# Check y layout

from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

##### **Note**: The Bayesian optimization function only accepts y data in “multiclass”.

### 3. Data Split 

In [63]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [64]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


### 4. Bayesian Hyperparameter Optimization

In [66]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [67]:
# First, let's properly encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Get the correct number of classes
n_classes = len(label_encoder.classes_)
print(f"Number of classes: {n_classes}")
print(f"Classes: {label_encoder.classes_}")

# Verify the new label range
print(f"Label range: {np.min(y_train_encoded)} to {np.max(y_train_encoded)}")

timesteps = X_train.shape[1]
input_dim = X_train.shape[2]
print(f"Timesteps: {timesteps}")
print(f"Classes: {input_dim}")

Number of classes: 14
Classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
Label range: 0 to 13
Timesteps: 15
Classes: 9


In [68]:
y_train_encoded.shape

(17212,)

In [69]:
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
                            layers1, layers2, normalization, dropout, dropout_rate):

    neurons = round(neurons)
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    activation = activationL[round(activation)]
    kernel = round(kernel)
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2) 

# Create model
    # def cnn_model():
        # Create model
    model = Sequential()
    
    # Add Conv1D layer
    model.add(Conv1D(filters=int(neurons), 
                     kernel_size=int(kernel), 
                     activation=activation,
                     input_shape=(timesteps, input_dim)))
    
    if normalization > 0.5:
            model.add(BatchNormalization())
    
        # Add Dense layers (first block)
    for _ in range(int(layers1)):
            model.add(Dense(int(neurons), activation=activation))
    
    model.add(Dropout(dropout_rate))
    
        # Add Dense layers (second block)
    for _ in range(int(layers2)):
            model.add(Dense(int(neurons), activation=activation))
    
    model.add(MaxPooling1D())
    model.add(Flatten())
        # Ensure the output layer has the correct number of nodes
    model.add(Dense(n_classes, activation='softmax'))
    
        # Compile model
    model.compile(optimizer=optimizer,
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
        # return model
 # The following lines of code were commented out because the cross validation is consistently come up with an error
    # es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    # nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
  
    # #score = cross_val_score(nn, X_train, y_train_encoded, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    # score = cross_val_score(estimator=nn, X=X_train, y=y_train_encoded, cv=5, fit_params={'sample_weight': weights}).mean()
    # return score


    # Define cross-validation strategy
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    scores = []
    
    # Perform cross-validation manually
    for train_idx, val_idx in kfold.split(X_train, y_train_encoded):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train_encoded[train_idx], y_train_encoded[val_idx]
        
        # Train the model
        history = model.fit(X_train_fold, y_train_fold,
                          batch_size=int(batch_size),
                          epochs=10,
                          validation_data=(X_val_fold, y_val_fold),
                          verbose=0)
        
        # Get the best validation accuracy
        val_acc = max(history.history['val_accuracy'])
        scores.append(val_acc)
    
    return np.mean(scores)
    
def objective(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
                            layers1, layers2, normalization, dropout, dropout_rate):
    try:
        return bay_area(
            neurons=neurons,
            activation=activation,
            kernel=kernel,
            optimizer=optimizer,
            learning_rate=learning_rate,
            batch_size=batch_size,
            epochs=epochs,
            layers1=layers1,
            layers2=layers2,
            normalization=normalization,
            dropout=dropout,
            dropout_rate=dropout_rate
        )
    except Exception as e:
        print(f"Error in model evaluation: {e}")
        return 0.0

In [70]:
# Define the parameter space
pbounds = {
    'neurons': (10, 100),
    'activation':(0, 9),    
    'kernel': (2, 5),
    'optimizer':(0,7),
    'learning_rate': (0.01, 1),
    'batch_size': (200, 1000),
    'epochs':(20, 50),    
    'layers1': (1, 3),
    'layers2': (1, 3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate': (0, 0.3)
}

# Initialize and run the optimization
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Start optimization
start_time = time.time()

try:
    optimizer.maximize(
        init_points=2,
        n_iter=14
    )
except Exception as e:
    print(f"Optimization failed with error: {e}")

print(f"Optimization took {(time.time() - start_time)/60:.2f} minutes")

# Print best results
if optimizer.max is not None:
    print("\nBest result:", optimizer.max)
    print("\nBest parameters:")
    for key, value in optimizer.max['params'].items():
        print(f"{key}: {value}")

# To convert predictions back to original labels later:
# original_labels = label_encoder.inverse_transform(predictions)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\sorca\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


| 1         | 0.644     | 3.371     | 960.6     | 0.732     | 0.1796    | 24.68     | 2.468     | 1.116     | 2.732     | 0.6051    | 73.73     | 0.02058   | 6.789     |


C:\Users\sorca\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


| 2         | 0.644     | 7.492     | 369.9     | 0.1818    | 0.05502   | 29.13     | 3.574     | 1.864     | 1.582     | 0.6157    | 22.55     | 0.2921    | 2.565     |


...

| 14        | 0.5764    | 7.73      | 203.9     | 0.7237    | 0.0316    | 36.75     | 2.66      | 2.132     | 2.851     | 0.4868    | 94.97     | 0.8837    | 3.464     |


C:\Users\sorca\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


| 15        | 0.7345    | 0.4992    | 205.6     | 0.6861    | 0.2889    | 40.56     | 3.571     | 1.308     | 1.523     | 0.2545    | 98.73     | 0.1117    | 3.814     |


C:\Users\sorca\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


| 16        | 0.6624    | 4.697     | 200.8     | 0.6473    | 0.06482   | 40.98     | 2.715     | 2.975     | 2.154     | 0.9745    | 99.52     | 0.3674    | 3.29      |
Optimization took 9.35 minutes

Best result: {'target': 0.9212804317474366, 'params': {'activation': 8.573725158270095, 'batch_size': 203.7843083366213, 'dropout': 0.3596828620945355, 'dropout_rate': 0.13106954502209853, 'epochs': 35.54009397985233, 'kernel': 2.391380993551486, 'layers1': 2.058084471812772, 'layers2': 1.1286824172695762, 'learning_rate': 0.26647435930571156, 'neurons': 99.11675828727839, 'normalization': 0.8224295254451733, 'optimizer': 2.143004695311215}}

Best parameters:
activation: 8.573725158270095
batch_size: 203.7843083366213
dropout: 0.3596828620945355
dropout_rate: 0.13106954502209853
epochs: 35.54009397985233
kernel: 2.391380993551486
layers1: 2.058084471812772
layers2: 1.1286824172695762
learning_rate: 0.26647435930571156
neurons: 99.11675828727839
normalization: 0.8224295254451733
optimizer

In [71]:
optimum = optimizer.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'relu',
 'batch_size': 204,
 'dropout': 0.3596828620945355,
 'dropout_rate': 0.13106954502209853,
 'epochs': 36,
 'kernel': 2.391380993551486,
 'layers1': 2,
 'layers2': 1,
 'learning_rate': 0.26647435930571156,
 'neurons': 99,
 'normalization': 0.8224295254451733,
 'optimizer': <keras.src.optimizers.rmsprop.RMSprop at 0x23618429460>}

### 5. Running the Model with Optimized Hyperparameters

In [82]:
# Set the model with optimized hyperparameters

epochs = optimum['epochs']
batch_size = optimum['batch_size']

timesteps = len(X_train[0])  
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = optimum['layers1']
layers2 = optimum['layers2']
activation = optimum['activation']
kernel = int(round(optimum['kernel']))  
neurons = optimum['neurons']
normalization = optimum['normalization']  
dropout = optimum['dropout']  
dropout_rate = optimum['dropout_rate']  
optimizer = RMSprop(optimum['learning_rate'])  

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

# Add second set of dense layers
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())  # Max-pooling layer to downsample the data
model.add(Flatten())  # Flatten for the dense layers
model.add(Dense(n_classes, activation='softmax'))  # Output layer for classification

# Compile the model with specified loss and optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [84]:
model.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_17 (Conv1D)              │ (None, 14, 99)         │         1,881 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 14, 99)         │           396 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_88 (Dense)                │ (None, 14, 99)         │         9,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_89 (Dense)                │ (None, 14, 99)         │         9,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_90 (Dense)                │ (None, 14, 99)         │         9,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_17 (MaxPooling1D) │ (None, 7, 99)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 693)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_91 (Dense)                │ (None, 15)             │        10,410 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,387 (165.57 KB)

 Trainable params: 42,189 (164.80 KB)

 Non-trainable params: 198 (792.00 B)

In [86]:
# Put the y_test set back into a one-hot configuration
y_train_one_hot = to_categorical(y_train_encoded, num_classes=15)

In [88]:
# Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [90]:
# Compile the model with categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [92]:
# Fit the model to the data
model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/36
85/85 - 2s - 23ms/step - accuracy: 0.6063 - loss: 6943.7642
Epoch 2/36
85/85 - 1s - 7ms/step - accuracy: 0.6440 - loss: 1.1757
Epoch 3/36
85/85 - 1s - 8ms/step - accuracy: 0.6440 - loss: 1.1736
Epoch 4/36
85/85 - 1s - 7ms/step - accuracy: 0.6440 - loss: 1.1735
Epoch 5/36
85/85 - 1s - 7ms/step - accuracy: 0.6440 - loss: 1.1748
Epoch 6/36
85/85 - 1s - 7ms/step - accuracy: 0.6440 - loss: 1.1749
Epoch 7/36
85/85 - 1s - 8ms/step - accuracy: 0.6440 - loss: 1.1750
Epoch 8/36
85/85 - 1s - 8ms/step - accuracy: 0.6440 - loss: 1.1744
Epoch 9/36
85/85 - 1s - 8ms/step - accuracy: 0.6440 - loss: 1.1752
Epoch 10/36
85/85 - 1s - 8ms/step - accuracy: 0.6440 - loss: 1.1731
Epoch 11/36
85/85 - 1s - 8ms/step - accuracy: 0.6440 - loss: 1.1723
Epoch 12/36
85/85 - 1s - 8ms/step - accuracy: 0.6440 - loss: 1.1735
Epoch 13/36
85/85 - 1s - 8ms/step - accuracy: 0.6440 - loss: 1.1755
Epoch 14/36
85/85 - 1s - 7ms/step - accuracy: 0.6440 - loss: 1.1752
Epoch 15/36
85/85 - 1s - 7ms/step - accuracy: 0.6440 

In [94]:
# Define list of stations names
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [96]:
print("Unique classes in y_test:", np.unique(y_test))

Unique classes in y_test: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]


In [98]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])


In [100]:
# Before making predictions, convert y_test to one-hot format
y_test_one_hot = to_categorical(y_test, num_classes=15)

# Predict the class probabilities
y_pred = model.predict(X_test)

# Convert y_test and y_pred to class labels
if y_test_one_hot.ndim == 1:
    y_test_labels = y_test_one_hot
else:
    y_test_labels = np.argmax(y_test_one_hot, axis=1)

if y_pred.ndim == 1:
    y_pred_labels = y_pred
else:
    y_pred_labels = np.argmax(y_pred, axis=1)

# Manually calculate accuracy
correct_predictions = np.sum(y_test_labels == y_pred_labels)
total_samples = len(y_test_labels)
accuracy = correct_predictions / total_samples

print(f'Accuracy: {accuracy * 100:.2f}%')

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 64.17%


In [102]:
y_pred_labels = np.argmax(y_pred, axis=1)

In [104]:
# Convert y_test_one_hot back to class labels
y_test_labels = np.argmax(y_test_one_hot, axis=1)

In [106]:
# Create Confusion Matrix
print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL
True             
BASEL        3682
BELGRADE     1092
BUDAPEST      214
DEBILT         82
DUSSELDORF     29
HEATHROW       82
KASSEL         11
LJUBLJANA      61
MAASTRICHT      9
MADRID        458
MUNCHENB        8
OSLO            5
STOCKHOLM       4
VALENTIA        1
